## Sentiment Analysis

### Import libraries

In [19]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

### 1. Load the TSV file

In [22]:
dataset = pd.read_csv("student_feedback.tsv", delimiter="\t", quoting=3)

### 2. Clean the text

In [25]:
nltk.download('stopwords')
corpus = []
ps = PorterStemmer()

for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Convert text into Bag-of-Words

In [28]:
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()

y = dataset.iloc[:, 1].values   # Assuming 2nd column = sentiment

### 4. Split into training/testing

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

### 5. Train Naive Bayes

In [34]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

### 6. Predict on test

In [37]:
y_pred = classifier.predict(X_test)

### 7. Evaluate

In [40]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7368421052631579
Confusion Matrix:
 [[4 1 0]
 [0 4 3]
 [0 1 6]]


### 8. Predict your own review

In [47]:
def predict_review(text):
    # Clean text
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    
    # Convert to bag of words
    vector = cv.transform([review]).toarray()
    
    # Predict
    result = classifier.predict(vector)
    return "Positive 😃" if result[0] == 1 else "Negative 😞"

# Example
print(predict_review("She is punctual and manages her duties responsibly."))

Positive 😃


Overall Model Insights

Best performance: Class 0 (Negative)

Worst performance: Class 1 (Positive)

Most confusion: Between Positive (1) and Neutral (2)

Accuracy: 73.68%

🔵 Row 0 → Actual class 0

[4 1 0]

Value	Meaning
4	Model predicted class 0 correctly 4 times
1	Model predicted class 1 incorrectly for a class-0 sample (mistake)
0	Model predicted class 2 incorrectly for a class-0 sample
🟡 Row 1 → Actual class 1

[0 4 3]

Value	Meaning
0	Model predicted class 0 incorrectly for a class-1 sample
4	Model predicted class 1 correctly 4 times
3	Model predicted class 2 incorrectly for a class-1 sample (biggest mistake area!)
🟢 Row 2 → Actual class 2

[0 1 6]

Value	Meaning
0	Model predicted class 0 incorrectly for a class-2 sample
1	Model predicted class 1 incorrectly for a class-2 sample
6	Model predicted class 2 correctly 6 times